In [2]:
import umap.aligned_umap
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
path1 = "/home/yifan/dataset/clean/pairflip/cifar10/0"
path2 = "/home/yifan/Exp/Dropout/0.3/experiment1"

In [3]:
def getData(path,epoch):
    ##### get representation
    X_train = np.load(os.path.join(path,"Model","Epoch_{}".format(epoch),"train_data.npy"))
    X_test = np.load(os.path.join(path,"Model","Epoch_{}".format(epoch),"test_data.npy"))
    
    #### get label
    training_label_loc =  os.path.join(path, "Training_data", "training_dataset_label.pth")
    y_train = torch.load(training_label_loc, map_location="cpu")
    y_train = y_train.numpy()
    testing_label_loc =  os.path.join(path, "Testing_data", "testing_dataset_label.pth") 
    y_test = torch.load(testing_label_loc, map_location="cpu")
    y_test = y_test.numpy()

    return X_train, X_test, y_train, y_test


In [4]:
X_train1, X_test1, y_train1, y_test1 = getData(path1,150)
X_train2, X_test2, y_train2, y_test2 = getData(path2,150)

In [5]:
X_train2 = X_train2.reshape(X_train2.shape[0],X_train2.shape[1])

In [6]:
import numpy as np
from scipy.linalg import svd

# 假设 data1 和 data2 已经给出
# 例如：
# data1 = np.random.rand(50000, 512)
# data2 = np.random.rand(50000, 512)

def orthogonal_mapping(data1, data2):
    """
    使用Orthogonal Mapping方法将data2映射到data1空间。
    
    :param data1: numpy array, shape (n_samples, n_features)
    :param data2: numpy array, shape (n_samples, n_features)
    :return: data2_mapped: numpy array, 映射后的data2
    """
    # 步骤1: 中心化数据
    data1_centered = data1 - np.mean(data1, axis=0)
    data2_centered = data2 - np.mean(data2, axis=0)

    # 步骤2: 计算交叉协方差矩阵
    C = data2_centered.T @ data1_centered

    # 步骤3: 奇异值分解
    U, _, Vt = svd(C)

    # 步骤4: 计算正交映射矩阵
    W = U @ Vt

    # 步骤5: 应用映射矩阵
    data2_mapped = data2_centered @ W
    
    return data2_mapped

# 使用函数
# data2_mapped = orthogonal_mapping(data1, data2)


In [7]:
print(X_train1.shape)
print(X_train2.shape)
data2_mapped = orthogonal_mapping(X_train1,X_train2)

(50000, 512)
(50000, 512)


In [8]:
np.random.seed(23423)
import sys
sys.path.append('/home/yifan/projects/cophi/ContraVis')
from eval.evaluate import *

In [9]:
val = evaluate_high_dimesion_trans_knn_preserving(X_train2,data2_mapped )
print("use orthogonal_mapping get transformed data that nn_perserving rate is: {}%".format(val*100) )

use orthogonal_mapping get transformed data that nn_perserving rate is: 100.0%
